In [ ]:
flainar_cls_table = '/mnt/d/mystuff/flainar/cat_flainar.xlsx'
flainar_cls_sheet = 'clsflainar'

flainar_cls_col = 'tipologia'

db = 'flainar'

In [ ]:
from glass.ng.rd import tbl_to_obj
from glass.g.wt.sql import df_to_db
from glass.ng.sql.q import q_to_ntbl

In [ ]:
# Read excel table with pandas

clsdf = tbl_to_obj(flainar_cls_table, sheet=flainar_cls_sheet)

clsdf.rename(columns={'natural_1' : 'natural'}, inplace=True)

In [ ]:
# Get interest keys
osmkeys = [c for c in clsdf.columns.values if c != flainar_cls_col]

In [ ]:
osmkeys

In [ ]:
# Send classsification data to the database
clstbl = df_to_db(db, clsdf, 'flainar_cls')

In [ ]:
feattbl = (
    "(SELECT polygoncatid FROM polygons_osmcat "
    "GROUP BY polygoncatid)"
)

In [ ]:
# General key value table with polygonid AS fid

featkv = (
    "SELECT mtbl.polygoncatid AS fid, jtbl.key, jtbl.value "
    "FROM polygons_osmcat AS mtbl "
    "INNER JOIN osmcategory AS jtbl "
    "ON mtbl.osmcategoryid = jtbl.osmcategoryid"
)

In [ ]:
qkv = [(
    f"(SELECT fid, foo.value AS {k} "
    f"FROM ({featkv}) AS foo "
    f"WHERE foo.key='{k}') AS qq{k}"
) for k in osmkeys]

In [ ]:
print(qkv[0])

In [ ]:
# Get osm features tables with osm keys as columns
selcols = [f"qq{k}.{k}" for k in osmkeys]

leftjoins = [(
    f"LEFT JOIN {qkv[i]} ON "
    f"feattbl.polygoncatid = qq{osmkeys[i]}.fid"
) for i in range(len(osmkeys))]

qkascol = (
    f"SELECT feattbl.*, {', '.join(selcols)} "
    f"FROM {feattbl} AS feattbl "
    f"{' '.join(leftjoins)}"
)

In [ ]:
osmtable = q_to_ntbl(db, 'osm_tbl', qkascol)

In [ ]:
# Get final table
fq = (
    f"SELECT mtbl.*, jtbl.{flainar_cls_col} "
    f"FROM {osmtable} AS mtbl "
    "LEFT JOIN flainar_cls AS jtbl "
    f"ON {' AND '.join([f'mtbl.{k} = jtbl.{k}' for k in osmkeys])}"
)

In [ ]:
fq